In [1]:
n = 30

weights = {
    1: 2,   2: 5,   3: 3,   4: 23,  5: 17,
    6: 8,   7: 12,  8: 1,   9: 4,  10: 5,
    11: 7,  12: 2,  13: 4,  14: 7,  15: 20,
    16: 5,  17: 22, 18: 7,  19: 4,  20: 1,
    21: 14, 22: 0,  23: 26, 24: 16, 25: 6,
    26: 15, 27: 8,  28: 4,  29: 8,  30: 4
}

durations = {
    1: 18,  2: 71,  3: 1,   4: 4,   5: 30,
    6: 97,  7: 73,  8: 83,  9: 34, 10: 22,
    11: 33, 12: 30, 13: 35, 14: 63, 15: 25,
    16: 84, 17: 87, 18: 83, 19: 46, 20: 63,
    21: 76, 22: 60, 23: 8,  24: 85, 25: 85,
    26: 26, 27: 83, 28: 86, 29: 36, 30: 82
}


ready_dates = {
    1: 4,     2: 28,    3: 85,    4: 69,    5: 373,
    6: 397,   7: 413,   8: 425,   9: 410,   10: 478,
    11: 696,  12: 768,  13: 812,  14: 902,  15: 903,
    16: 906,  17: 1012, 18: 1113, 19: 1163, 20: 1079,
    21: 1335, 22: 1294, 23: 1330, 24: 1497, 25: 1432,
    26: 1587, 27: 1338, 28: 1341, 29: 929,  30: 982
}


due_dates = {
    1: 83,    2: 619,   3: 649,   4: 724,   5: 1117,
    6: 565,   7: 1237,  8: 593,   9: 1001, 10: 677,
    11: 1370, 12: 1167, 13: 1348, 14: 1175, 15: 1000,
    16: 1212, 17: 1238, 18: 1990, 19: 1908, 20: 1323,
    21: 1915, 22: 2143, 23: 2037, 24: 2135, 25: 1981,
    26: 1870, 27: 2029, 28: 1641, 29: 1437, 30: 1084
}

deadlines = {
    1: 2201,  2: 1270,  3: 1213,  4: 2356,  5: 2392,
    6: 636,   7: 1773,  8: 1362,  9: 3116, 10: 2786,
    11: 2037, 12: 2393, 13: 3582, 14: 1627, 15: 2062,
    16: 3511, 17: 3599, 18: 4220, 19: 2048, 20: 2703,
    21: 2856, 22: 3939, 23: 3812, 24: 3881, 25: 3363,
    26: 3804, 27: 2312, 28: 3626, 29: 3426, 30: 2384
}


successors = {
    1: [2],
    2: [8, 7, 3],
    3: [11, 6],
    4: [8, 6],
    5: [6],
    6: [19, 12, 9],
    7: [19, 12, 9],
    8: [19, 9],
    9: [23, 17, 15, 14, 13],
    10: [22, 19, 17, 14, 13],
    11: [23, 19, 17, 13],
    12: [23, 16, 13],
    13: [21, 20, 18],
    14: [21, 20, 18],
    15: [22, 20, 18],
    16: [22, 21, 18],
    17: [21, 18],
    18: [28, 25, 24],
    19: [24, 21],
    20: [25, 24],
    21: [27, 25],
    22: [30, 24],
    23: [30, 24],
    24: [27, 26],
    25: [30, 26],
    26: [29],
    27: [29],
    28: [29],
    29: [],
    30: []
}


In [2]:
predecessors = {i: [] for i in range(1, n+1)}
for i in range(1, n+1):
    for s in successors[i]:
        predecessors[s].append(i)


In [3]:
from collections import deque

indeg = {i: len(predecessors[i]) for i in range(1, n+1)}
q = deque([i for i in range(1, n+1) if indeg[i] == 0])
topo = []

while q:
    u = q.popleft()
    topo.append(u)
    for v in successors[u]:
        indeg[v] -= 1
        if indeg[v] == 0:
            q.append(v)

if len(topo) != n:
    raise ValueError("Precedence graph has a cycle!")


In [4]:
new_ready_dates = {i: ready_dates[i] for i in range(1, n+1)}
for i in topo:
    if predecessors[i]:
        new_ready_dates[i] = max(ready_dates[i], max(new_ready_dates[j] + durations[j] for j in predecessors[i]))

In [5]:
new_deadlines = {i: deadlines[i] for i in range(1, n+1)}

for i in reversed(topo):
    if successors[i]:
        new_deadlines[i] = min(new_deadlines[i], min(new_deadlines[s] - durations[s] for s in successors[i]))

In [10]:
schedule_solution = [
    {"job": 1, "start": 4, "end": 22},
    {"job": 2, "start": 28, "end": 99},
    {"job": 3, "start": 99, "end": 100},
    {"job": 4, "start": 100, "end": 104},
    {"job": 5, "start": 373, "end": 403},
    {"job": 6, "start": 403, "end": 500},
    {"job": 8, "start": 500, "end": 583},
    {"job": 7, "start": 583, "end": 656},
    {"job": 10, "start": 656, "end": 678},
    {"job": 9, "start": 678, "end": 712},
    {"job": 11, "start": 712, "end": 745},
    {"job": 12, "start": 768, "end": 798},
    {"job": 13, "start": 812, "end": 847},
    {"job": 14, "start": 902, "end": 965},
    {"job": 15, "start": 965, "end": 990},
    {"job": 16, "start": 990, "end": 1074},
    {"job": 20, "start": 1079, "end": 1142},
    {"job": 17, "start": 1142, "end": 1229},
    {"job": 19, "start": 1229, "end": 1275},
    {"job": 18, "start": 1275, "end": 1358},
    {"job": 21, "start": 1358, "end": 1434},
    {"job": 25, "start": 1434, "end": 1519},
    {"job": 28, "start": 1519, "end": 1605},
    {"job": 23, "start": 1605, "end": 1613},
    {"job": 22, "start": 1613, "end": 1673},
    {"job": 30, "start": 1673, "end": 1755},
    {"job": 24, "start": 1755, "end": 1840},
    {"job": 27, "start": 1840, "end": 1923},
    {"job": 26, "start": 1923, "end": 1949},
    {"job": 29, "start": 1949, "end": 1985}
]

# paste cả schedule_solution, durations, ready_dates, successors, deadlines như bạn đã có
# rồi chạy hàm dưới đây

def check_schedule(schedule, durations, ready_dates, successors, deadlines):
    # convert list -> dict
    start = {item["job"]: item["start"] for item in schedule}
    end   = {item["job"]: item["end"]   for item in schedule}
    jobs = set(durations.keys())
    sched_jobs = set(start.keys())

    # 1. đủ job không thừa/thiếu
    if jobs != sched_jobs:
        missing = sorted(list(jobs - sched_jobs))
        extra   = sorted(list(sched_jobs - jobs))
        return False, f"Lỗi: thiếu job {missing}, thừa job {extra}"

    # 2. start < end và đúng duration
    for j in jobs:
        if end[j] - start[j] != durations[j]:
            return False, f"Lỗi: job {j} sai duration: {end[j]-start[j]} != {durations[j]}"

    # 3. ready time
    for j in jobs:
        if start[j] < ready_dates[j]:
            return False, f"Lỗi: job {j} start {start[j]} < ready_time {ready_dates[j]}"

    # 4. precedence (đảm bảo end[pred] <= start[succ])
    for pred, succs in successors.items():
        for s in succs:
            if end[pred] > start[s]:
                return False, f"Lỗi precedence: tiền-tử {pred} kết thúc {end[pred]} > start của hậu-tử {s} = {start[s]}"

    # 5. single-machine (không overlap)
    intervals = sorted([(start[j], end[j], j) for j in jobs])
    for i in range(len(intervals) - 1):
        s1, e1, j1 = intervals[i]
        s2, e2, j2 = intervals[i+1]
        # nếu e1 > s2 thì overlap (kể cả chồng nhau tại điểm thời gian)
        if e1 > s2:
            return False, f"Overlap: job {j1} ({s1}-{e1}) overlap job {j2} ({s2}-{e2})"

    # 6. deadline (nếu có) — kiểm tra end <= deadline
    viols = []
    for j in sorted(jobs):
        if end[j] > deadlines[j]:
            viols.append((j, end[j], deadlines[j]))
    if viols:
        return False, f"Deadline violations (job, end, deadline): {viols}"

    return True, "Lịch hợp lệ: thỏa tất cả ràng buộc cứng (bao gồm deadline nếu cung cấp)"

# Ví dụ chạy:
ok, msg = check_schedule(schedule_solution, durations, new_ready_dates, successors, new_deadlines)
print(ok, msg)


True Lịch hợp lệ: thỏa tất cả ràng buộc cứng (bao gồm deadline nếu cung cấp)


In [ ]:
# cplex_schedule_with_A.py
from docplex.mp.model import Model

def solve_with_cplex_A(n, durations, ready_dates, deadlines, successors):
    mdl = Model("single_machine_with_A")

    jobs = list(range(1, n+1))
    t_min = min(ready_dates.values())
    t_max = max(deadlines.values())  # maximum time horizon

    # -----------------------------
    # VARIABLES
    # -----------------------------

    # S[i,t] = 1 if job i starts at time t
    S = {}
    for i in jobs:
        for t in range(ready_dates[i], deadlines[i] - durations[i] + 1):
            S[(i,t)] = mdl.binary_var(name=f"S_{i}_{t}")

    # A[i,t] = 1 if job i is active at time t
    A = {}
    for i in jobs:
        for t in range(ready_dates[i], deadlines[i]):
            A[(i,t)] = mdl.binary_var(name=f"A_{i}_{t}")

    # -----------------------------
    # CONSTRAINTS
    # -----------------------------

    # 1) Each job must start exactly once
    for i in jobs:
        mdl.add_constraint(
            mdl.sum(S[(i,t)] for t in range(ready_dates[i], deadlines[i] - durations[i] + 1)) == 1,
            ctname=f"start_once_{i}"
        )

    # 2) Activation: S(i,t_start) → A(i,t) for t in [t_start, t_start + pi - 1]
    for i in jobs:
        p = durations[i]
        for t_start in range(ready_dates[i], deadlines[i] - p + 1):
            for tau in range(t_start, t_start + p):
                mdl.add_constraint(
                    A[(i, tau)] >= S[(i, t_start)],
                    ctname=f"activate_{i}_{t_start}_{tau}"
                )

    # 3) Machine capacity: at most 1 active at each time t
    for t in range(t_min, t_max):
        mdl.add_constraint(
            mdl.sum(A[(i,t)] for i in jobs if (i,t) in A) <= 1,
            ctname=f"capacity_{t}"
        )

    # 4) Precedence constraints using A or S
    # For i -> j: job j cannot start before i finishes.
    for i in jobs:
        p_i = durations[i]
        for j in successors.get(i, []):
            # s_j >= s_i + p_i encoded as:
            # For every start t_j of j: forbid if < t_i+p_i
            for t_i in range(ready_dates[i], deadlines[i] - p_i + 1):
                finish_i = t_i + p_i
                for t_j in range(ready_dates[j], deadlines[j] - durations[j] + 1):
                    if t_j < finish_i:
                        # Forbidden pattern: i starts at t_i AND j starts at t_j
                        mdl.add_constraint(
                            S[(i, t_i)] + S[(j, t_j)] <= 1,
                            ctname=f"prec_{i}_{j}_{t_i}_{t_j}"
                        )

    # (optional) Cmax for objective
    Cmax = mdl.integer_var(lb=0, ub=t_max + 100, name="Cmax")
    for i in jobs:
        p = durations[i]
        mdl.add_constraint(
            Cmax >= mdl.sum((t + p) * S[(i,t)]
                            for t in range(ready_dates[i], deadlines[i] - p + 1))
        )

    mdl.minimize(Cmax)

    # -----------------------------
    # SOLVE
    # -----------------------------
    sol = mdl.solve(log_output=True)
    if sol is None:
        print("No feasible solution.")
        return None

    # Extract schedule
    schedule = {}
    for i in jobs:
        for t in range(ready_dates[i], deadlines[i] - durations[i] + 1):
            if sol.get_value(S[(i,t)]) > 0.5:
                schedule[i] = t

    print("\nSchedule:")
    for i in sorted(schedule, key=lambda x: schedule[x]):
        print(f"Job {i}: start={schedule[i]}, end={schedule[i] + durations[i]}")

    return schedule

solve_with_cplex_A(n, durations, new_ready_dates, new_deadlines, successors)


Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Presolve has eliminated 1005 rows and 60 columns...
Presolve has improved bounds 3385 times...
Aggregator has done 564 substitutions...
Tried aggregator 3 times.
